In [1]:
# ACKNOWLEDGEMENTS:
# Code copied and modified (MIT License) from 
# (https://github.com/JanZrimec/DeepExpression/blob/master/scripts/Chapter_3_1.ipynb)
# Original author: Jan Zrimec (https://github.com/JanZrimec)

import numpy as np
import pandas as pd

from scipy import stats, special, spatial

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.lines import Line2D
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import importlib.util
import os

import warnings
warnings.filterwarnings('ignore')

#s.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
## datasets
folder = '../data/'

# Data
fn1 = 'scerevisiae_tpm_full_cut5.csv'
fn2 = 'scerevisiae_accessions.tsv'
fn3 = 'scerevisiae.rsd1.lmbda_22.npz'
fn4 = 'Saccharomyces_cerevisiae.R64-1-1.regions.csv'

fn5 = 'organisms_deep_results.csv'

fname_data1 = folder+'scerevisiae.rsd1.lmbda_22.npz'
fname_module1 = folder+'Model_C3F2all.py'
fname_p1 = folder+'scerevisiae_good_rsd1_lmbda22_Model_C3F2all_1234_c008090fd8e4b9971fe71eed502c9f7b_best.p'
fname_weights1 = folder+'scerevisiae_good_rsd1_lmbda22_Model_C3F2all_1234_c008090fd8e4b9971fe71eed502c9f7b_best'

fn6 = 'df_Keren2013_predictions.csv'
fn7 = 'df_Yamanishi2013_predictions.csv'


In [3]:
def load_data(fname):
    # X is multi-variable array
    # Y contains single variable - fix shape for Keras

    npzfile = np.load(fname,allow_pickle=True)
    Xh_train = npzfile['arr_0']
    Xh_test = npzfile['arr_1']
    Xv_train = npzfile['arr_2'][:,8:]
    Xv_test = npzfile['arr_3'][:,8:]
    Y_train = npzfile['arr_4']
    Y_test = npzfile['arr_5']
    names_train = npzfile['arr_6']
    names_test = npzfile['arr_7']

    X_train = list()
    X_train.append(Xh_train)
    #X_train.append(Xv_train)
    X_test = list()
    X_test.append(Xh_test)
    #X_test.append(Xv_test)

    Y_train = Y_train.astype(np.float32).reshape((-1,))
    Y_test = Y_test.astype(np.float32).reshape((-1,))

    return (np.concatenate((Y_train, Y_test)),
            np.concatenate((names_train,names_test)),
            np.concatenate((Xv_train,Xv_test)))

# load modules - POC model
def load_module(model_path):
    '''loads module containing models given path'''
    spec = importlib.util.spec_from_file_location('module',model_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module


In [4]:
## load data and model

module = load_module(fname_module1)

# load p
p = pd.read_csv(fname_p1,sep=';')
p = p.iloc[0]
p.drop(['path','overfit'],inplace=True)
print(p.shape)

# load data
X_train, X_test, Y_train, Y_test = module.load_data(fname_data1)
# Y = np.concatenate((Y_train, Y_test))
# X = np.concatenate((X_train[0], X_test[0]))
print(len(X_test))
print(len(Y_test))

# model
input_shapes = [sl.shape[1:] for sl in X_train]
model = module.POC_model(input_shapes, p)
# set weights model 1
print("Loading model 1 from disk..")
model.load_weights(fname_weights1)


(42,)
2
425
Loading model 1 from disk..


In [5]:
file_yeast_mod_npz = "/home/andy/Saccharomyces_cerevisiae_R64-1-1.Sacce1_MODIFIED.npz"

working_npz = np.load(file_yeast_mod_npz, allow_pickle=True)
X_hot = working_npz["og_hot"]
X_var = working_npz["og_var"]

current_input = list()
current_input.append(X_hot)
current_input.append(X_var)
print("loaded input")

loaded input


In [6]:
Y_pred = model.predict(current_input)
print("predictions done")

predictions done


In [7]:
npy_out_file = "/home/andy/predictions_YEAST/Saccharomyces_cerevisiae_R64-1-1.Sacce1_MODIFIED.npy"
np.save(npy_out_file, Y_pred)
print("saved")

saved


In [8]:
test_Y_pred = np.load(npy_out_file, allow_pickle=True)
print(test_Y_pred)

[[7.553082 ]
 [6.9116087]
 [7.3022065]
 [6.5957522]
 [7.3093743]
 [7.177694 ]
 [7.221905 ]
 [6.7287602]
 [7.7012544]
 [7.2830176]
 [6.992248 ]
 [7.443007 ]
 [6.9153857]
 [7.402662 ]
 [6.856152 ]
 [7.158108 ]
 [5.0509086]
 [6.9954476]
 [4.6570554]
 [7.0427065]
 [7.0127606]
 [6.6560626]
 [7.037621 ]
 [7.483103 ]
 [6.913189 ]
 [7.0300207]
 [7.0227284]
 [6.4306555]
 [6.9369693]
 [7.46388  ]
 [6.941281 ]
 [6.2674813]
 [7.1559615]
 [6.8605914]
 [6.82406  ]
 [6.495995 ]
 [7.709535 ]
 [7.382441 ]
 [6.5222588]
 [6.680422 ]
 [6.4483113]
 [6.574367 ]
 [7.1972194]
 [6.689777 ]
 [7.599898 ]
 [6.73039  ]
 [7.07182  ]
 [8.2075205]
 [6.7622895]
 [4.5139756]
 [7.175612 ]
 [7.6852202]
 [7.4207716]
 [7.5367804]
 [7.2091494]
 [7.6242533]
 [7.2060537]
 [7.574801 ]
 [7.0815654]
 [7.542887 ]
 [7.4088225]
 [6.838036 ]
 [7.7127957]
 [7.2258306]
 [7.2449307]
 [6.7887564]
 [6.875079 ]
 [6.7175174]
 [7.533933 ]
 [6.9860473]
 [7.202639 ]
 [6.8311357]
 [7.5613275]
 [7.5117655]
 [7.341186 ]
 [5.730389 ]
 [5.9482965]

In [9]:
print(len(test_Y_pred))

425
